In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
#영화 리뷰 데이터, 긍정 혹은 부정으로 이진분류 되어있는 데이터(클래스가 두 개인 데이터)
dataset, info = tfds.load("imdb_reviews/subwords8k", with_info = True, as_supervised = True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [3]:
#텍스트는 토큰 단위로 매칭 되어있음, 그러므로 encoder에 텍스트의 매칭되는 값들 저장
encoder = info.features['text'].encoder

In [4]:
#vocabulary의 size확인
print('Voca size: {}'.format(encoder.vocab_size))

Voca size: 8185


In [5]:
#string을 인코딩, 디코딩해서 벡터 확인 -> 인코딩과 디코딩은 가역적임
sample_string = 'Hello Tensorflow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string is {}'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 2934, 7975]
The original string is Hello Tensorflow.


In [6]:
#토큰 단위가 어떻게 매칭되는지 확인

for idx in encoded_string:
  print('{} ----> {}'.format(idx, encoder.decode([idx])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
2934 ----> flow
7975 ----> .


# Data spliting

In [7]:
#인코딩된 문자열을 학습, 테스트 값으로 분할
buffer_size = 10000
batch_size = 64

train_dataset = train_dataset.shuffle(buffer_size)
train_dataset = train_dataset.padded_batch(batch_size)

test_dataset = test_dataset.padded_batch(batch_size)

# RNN

In [8]:
#모델 -> 임베딩 벡터, 양방향 RNN 레이어, Dense 레이어로 구성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1)
])


In [9]:
# loss는 binary cross-entropy, optimizer는 Adam optimizer
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              optimizer = tf.keras.optimizers.Adam(1e-4),
              metrics = ['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              16512     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 548,673
Trainable params: 548,673
Non-trainable params: 0
_________________________________________________________________


In [13]:
#모델의 학습 진행
model.fit(train_dataset, epochs = 10, 
          validation_data = test_dataset,
          validation_steps = 30)


Epoch 1/10
391/391 [==============================] - 494s 1s/step - loss: 0.6557 - accuracy: 0.5481 - val_loss: 0.6893 - val_accuracy: 0.4917
Epoch 2/10
391/391 [==============================] - 494s 1s/step - loss: 0.6654 - accuracy: 0.5217 - val_loss: 0.5253 - val_accuracy: 0.6995
Epoch 3/10
391/391 [==============================] - 491s 1s/step - loss: 0.4257 - accuracy: 0.8062 - val_loss: 0.4215 - val_accuracy: 0.8042
Epoch 4/10
391/391 [==============================] - 489s 1s/step - loss: 0.3028 - accuracy: 0.8766 - val_loss: 0.4002 - val_accuracy: 0.8193
Epoch 5/10
391/391 [==============================] - 493s 1s/step - loss: 0.2288 - accuracy: 0.9100 - val_loss: 0.3734 - val_accuracy: 0.8562
Epoch 6/10
391/391 [==============================] - 489s 1s/step - loss: 0.1804 - accuracy: 0.9359 - val_loss: 0.3938 - val_accuracy: 0.8568
Epoch 7/10
391/391 [==============================] - 489s 1s/step - loss: 0.1341 - accuracy: 0.9533 - val_loss: 0.4286 - val_accuracy: 0.8375

In [14]:
#test data에 대한 정확도와 loss 파악
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 79s 202ms/step - loss: 0.6378 - accuracy: 0.8246
Test Loss: 0.6378470063209534
Test Accuracy: 0.8246399760246277


# LSTM

In [15]:
#RNN은 장기기억에 단점을 가지고 있음 -> vanishing gradient 발생 -> LSTM으로 보완을 해봄
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1)
])


In [16]:
# loss는 binary cross-entropy, optimizer는 Adam optimizer
model1.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              optimizer = tf.keras.optimizers.Adam(1e-4),
              metrics = ['accuracy'])

In [17]:
model1.fit(train_dataset, epochs = 10, 
           validation_data = test_dataset, 
           validation_steps = 30)

Epoch 1/10
391/391 [==============================] - 986s 3s/step - loss: 0.6557 - accuracy: 0.5466 - val_loss: 0.5005 - val_accuracy: 0.7896
Epoch 2/10
391/391 [==============================] - 977s 2s/step - loss: 0.3561 - accuracy: 0.8507 - val_loss: 0.3442 - val_accuracy: 0.8708
Epoch 3/10
391/391 [==============================] - 978s 3s/step - loss: 0.2522 - accuracy: 0.9029 - val_loss: 0.3603 - val_accuracy: 0.8698
Epoch 4/10
391/391 [==============================] - 972s 2s/step - loss: 0.2175 - accuracy: 0.9180 - val_loss: 0.3192 - val_accuracy: 0.8672
Epoch 5/10
391/391 [==============================] - 969s 2s/step - loss: 0.1896 - accuracy: 0.9314 - val_loss: 0.3290 - val_accuracy: 0.8635
Epoch 6/10
391/391 [==============================] - 964s 2s/step - loss: 0.1726 - accuracy: 0.9376 - val_loss: 0.3484 - val_accuracy: 0.8698
Epoch 7/10
391/391 [==============================] - 971s 2s/step - loss: 0.1450 - accuracy: 0.9501 - val_loss: 0.3737 - val_accuracy: 0.8646

In [18]:
#test data에 대한 정확도와 loss 파악
test_loss, test_acc = model1.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 227s 582ms/step - loss: 0.4657 - accuracy: 0.8572
Test Loss: 0.46565738320350647
Test Accuracy: 0.8571599721908569
